In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Research/FU_AI_LAB/Ear_recognize/"

/content/drive/My Drive/Research/FU_AI_LAB/Ear_recognize


In [3]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm_notebook as tqdm
import time
import pickle
tqdm().pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

In [4]:
CATE_DATASET_PATH = "./Dataset/EarVN1.0dataset/Images/"

In [5]:
categories = os.listdir(CATE_DATASET_PATH)
print(categories)

['029.Ho_Trung_D', '081.Thanh_D', '024.Gin_Tuan_K', '122.La_Thanh_H', '002.LeDuong_BL', '155.Truong_Quynh_A', '109.Go_Joon_H', '075.Rym', '162.Vu_Cat_T', '114.Kieu_Tr', '052.Luong_Bang_Q', '135.Minh_T', '026.Hac_Hoa_K', '014.Dan_Ng', '083.ThanhTr', '020.Duong_D', '137.My_T', '095.Tung_Dg', '132.Mid_Ng', '007.Chau_KP', '149.Thuy_H', '099.Amber', '104.Chau_B', '110.H_Hen_N', '008.Chi_D', '143.Pham_Thanh_H', '050.Long_Nh', '009.Chi_Th', '028.Ho_Quang_H', '153.Trang_Ph', '051.Lou_Ho', '017.Dao_Ba_L', '043.L_HA', '121.Kim_Yoo_J', '021.Dustin_Phuc_Ng', '103.Cha_Mi', '087.Tran Th', '064.Noo_Phuoc_Th', '036.Jun_Ph', '035.Isaac', '065.Ong_Cao_Th', '154.Trieu_Le_D', '116.Kim_Ji_W', '030.Hoai_L', '078.ST', '161.Vo_Hoang_Y', '088.Trinh_Thang_B', '152.Trang_K', '086.Tran Dinh Q', '133.Minh_H', '038.Kenvin_Kh', '150.Toc_T', '076.Sky_ST', '130.Mai_Ng', '163.Xa_Thi_M', '040.Khuong_Ng', '013.Dam_Vinh_H', '010.Chu_B', '047.Lam_Vinh_H', '037.Justatee', '012.Dai_Nhan', '059.Ngo_Kien_H', '140.Nguyen_Thi_Tr

In [6]:
IMG_HEIGHT = 257
IMG_WIDTH = 161
BATCH_SIZE = 32

training_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    CATE_DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size =(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    CATE_DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size =(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

Found 28422 files belonging to 164 classes.
Using 22738 files for training.
Found 28422 files belonging to 164 classes.
Using 5684 files for validation.


In [7]:
# for image_batch, labels_batch in training_dataset:
#     print(image_batch.shape)
#     print(labels_batch.shape)

#     break

In [8]:
param_grid={
    'C':[0.1,1,100],
    'gamma':[0.0001,0.001],
    'kernel':['rbf','poly']
    }

In [9]:
svc = svm.SVC(probability=True)

In [10]:
len(training_dataset)

711

In [ ]:
%%capture

batch_index = 0
with tqdm(total=len(training_dataset), desc ="train") as pbar:
    for image_batch, labels_batch in training_dataset:
        print(image_batch.shape)
        print(labels_batch.shape)
        batch_index = batch_index + 1
        print("batch_index: ", batch_index)

        flat_data_arr = []
        target_arr = []

        with tqdm(total=BATCH_SIZE, desc ="convert_tf_to_numpy") as psubbar:
            for i in range(BATCH_SIZE):
                flat_data = np.array(image_batch).flatten()
                target = np.array(labels_batch)

                flat_data_arr.append(flat_data)
                target_arr.append(target)
                psubbar.update(1)


        df = pd.DataFrame(flat_data_arr)
        df['Target'] = target_arr

        x = df.iloc[:,:-1]
        y = df.iloc[:,-1]

        if batch_index == 1:
            model = GridSearchCV(svc,param_grid)

        model.fit(x,y)
        del df
        pbar.update(1)

pickle.dump(model,open('img_model.p','wb'))
print("Pickle is dumped successfully")

In [ ]:
flat_data_arr=[] #input array
target_arr=[] #output array

for i in categories:
    print(f'loading... category : {i}')
    path=os.path.join(CATE_DATASET_PATH,i)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(257,161,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(categories.index(i))
        del img_array
        del img_resized
    print(f'loaded category:{i} successfully')

loading... category : 029.Ho_Trung_D
loaded category:029.Ho_Trung_D successfully
loading... category : 081.Thanh_D
loaded category:081.Thanh_D successfully
loading... category : 024.Gin_Tuan_K
loaded category:024.Gin_Tuan_K successfully
loading... category : 122.La_Thanh_H
loaded category:122.La_Thanh_H successfully
loading... category : 002.LeDuong_BL
loaded category:002.LeDuong_BL successfully
loading... category : 155.Truong_Quynh_A
loaded category:155.Truong_Quynh_A successfully
loading... category : 109.Go_Joon_H
loaded category:109.Go_Joon_H successfully
loading... category : 075.Rym
loaded category:075.Rym successfully
loading... category : 162.Vu_Cat_T
loaded category:162.Vu_Cat_T successfully
loading... category : 114.Kieu_Tr
loaded category:114.Kieu_Tr successfully
loading... category : 052.Luong_Bang_Q
loaded category:052.Luong_Bang_Q successfully
loading... category : 135.Minh_T
loaded category:135.Minh_T successfully
loading... category : 026.Hac_Hoa_K
loaded category:026.